# Spectral Simulation on Sphere

<!-- SUMMARY: Simulations performed on a Sphere of the Spectral Model -->

<!-- CATEGORY: Methodology -->

Ce notebook est un résumé du notebook situé ici 

https://chart-studio.plotly.com/~empet/14813/heatmap-plot-on-a-spherical-map/#/

et se concentre sur le traçage des frontières (plus généralement d'un polygone) sur la sphère.

Il nécessite l'installation de **mpl_toolkits** qui peut être assez complexe.

Sous linux, sous réserve que la librairie GEOS soit installée, il suffit d'effectuer les commandes suivantes (le dossier à télécharger est assez lourd et la procédure peut prendre quelques minutes) :

En cas de problème, voir les détails ici :

https://matplotlib.org/basemap/users/installing.html .


In [ ]:
import numpy as np     
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import IPython
import gstlearn as gl
import gstlearn.document as gdoc
import gstlearn.plot3D as gop
import os

gdoc.setNoScroll()

In [ ]:
gl.defineDefaultSpace(gl.ESpaceType.SN)

mesh = gl.MeshSphericalExt()
mesh.resetFromDb(None,None,triswitch = "-r5",verbose=False)

model = gl.Model.createFromParam(gl.ECov.MATERN,range=0.2,param=1)

simu = gl.SimuSphericalParam(0,1000)
simu.display()
result = gl.simsph_mesh(mesh,model,simu,seed=23234,verbose=True)

In [ ]:
simu = gop.SurfaceOnMesh(mesh, intensity=result, opacity=1, showlegend=True)

In [ ]:
name = gdoc.loadData("boundaries", "world.poly")
poly = gl.Polygons.createFromNF(name)
boundaries = gop.PolygonOnSphere(poly)

In [ ]:
fig = go.Figure(data = [simu, boundaries])
f = fig.show()